# 🎭 Facial Expression Detection - Training Notebook

**Versi Khusus untuk Dataset dengan Class Imbalance Ekstrem**

Dataset Anda memiliki ketidakseimbangan kelas yang parah:
- jijik: 436 gambar
- senang: 7215 gambar

Notebook ini menggunakan **Focal Loss** dan **Class Weights Agresif** untuk mengatasi masalah ini.

## 1. Setup Environment

In [ ]:
!pip install "numpy<2.0" --force-reinstall --upgrade
!pip install tensorflow opencv-python matplotlib scikit-learn

In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dropout, Flatten, 
    Dense, BatchNormalization, Activation
)
from tensorflow.keras.callbacks import (
    ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
)
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight

import warnings
warnings.filterwarnings('ignore')

## 2. Focal Loss (untuk Class Imbalance)

In [ ]:
# --- FOCAL LOSS ---
# Memberikan penalti LEBIH BESAR untuk kesalahan di kelas minoritas

def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
        
        cross_entropy = -y_true * tf.math.log(y_pred)
        weight = alpha * y_true * tf.pow(1 - y_pred, gamma)
        focal_loss = weight * cross_entropy
        
        return tf.reduce_sum(focal_loss, axis=-1)
    return focal_loss_fixed

## 3. Konfigurasi

In [ ]:
IS_KAGGLE = os.path.exists('/kaggle/input')

if IS_KAGGLE:
    DATASET_DIR = '/kaggle/input/ekspresi-wajah1/dataset'
    TRAIN_DIR = os.path.join(DATASET_DIR, 'train')
    VALIDATION_DIR = os.path.join(DATASET_DIR, 'validation')
    MODELS_DIR = '/kaggle/working/models'
else:
    DATASET_DIR = 'dataset'
    TRAIN_DIR = os.path.join(DATASET_DIR, 'train')
    VALIDATION_DIR = os.path.join(DATASET_DIR, 'validation')
    MODELS_DIR = 'models'

os.makedirs(MODELS_DIR, exist_ok=True)
MODEL_PATH = os.path.join(MODELS_DIR, 'expression_model.h5')
LABELS_PATH = os.path.join(MODELS_DIR, 'class_labels.json')

INPUT_SHAPE = (48, 48, 1)
NUM_CLASSES = 7

print(f"Environment: {'Kaggle' if IS_KAGGLE else 'Local'}")

## 4. Data Generators (dengan filter .gitkeep)

In [ ]:
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator

def create_data_generators(train_dir, validation_dir, batch_size=64):
    # Augmentasi untuk meningkatkan variasi kelas minoritas
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        width_shift_range=0.15,
        height_shift_range=0.15,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    validation_datagen = ImageDataGenerator(rescale=1./255)
    
    # PENTING: classes parameter untuk EXCLUDE .gitkeep
    valid_classes = ['marah', 'jijik', 'takut', 'senang', 'netral', 'sedih', 'kaget']
    
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48, 48),
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='categorical',
        classes=valid_classes,
        shuffle=True
    )
    
    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(48, 48),
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='categorical',
        classes=valid_classes,
        shuffle=False
    )
    
    return train_generator, validation_generator, valid_classes

## 5. Model (Custom CNN)

In [ ]:
def create_cnn(input_shape=(48, 48, 1), num_classes=7):
    model = Sequential(name='FER_CNN')

    # Block 1
    model.add(Conv2D(64, (3, 3), padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # Block 2
    model.add(Conv2D(128, (5, 5), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # Block 3
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # Block 4
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # Dense
    model.add(Flatten())
    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Dense(num_classes, activation='softmax'))
    return model

## 6. Training

In [ ]:
# --- MAIN TRAINING ---

EPOCHS = 80
BATCH_SIZE = 64
LR = 0.0005

if not os.path.exists(TRAIN_DIR):
    print(f"Error: {TRAIN_DIR} tidak ditemukan!")
else:
    # 1. Generators
    train_gen, val_gen, class_names = create_data_generators(TRAIN_DIR, VALIDATION_DIR, BATCH_SIZE)
    
    print(f"\nClasses: {class_names}")
    print(f"Train: {train_gen.n} | Val: {val_gen.n}")
    
    # 2. Compute Class Weights (AGRESIF)
    print("\nMenghitung Class Weights...")
    class_counts = {}
    for name in class_names:
        path = os.path.join(train_gen.directory, name)
        count = len([f for f in os.listdir(path) if f.lower().endswith(('.jpg', '.png'))])
        class_counts[name] = count
        print(f"  {name}: {count}")
    
    labels = []
    for idx, name in enumerate(class_names):
        labels.extend([idx] * class_counts[name])
    
    weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
    class_weights = {i: w for i, w in enumerate(weights)}
    print(f"\nClass Weights: {class_weights}")
    
    # 3. Create Model
    print("\nMembangun Model...")
    model = create_cnn(INPUT_SHAPE, NUM_CLASSES)
    
    # Compile dengan FOCAL LOSS
    model.compile(
        optimizer=Adam(learning_rate=LR),
        loss=focal_loss(gamma=2.0, alpha=0.25),
        metrics=['accuracy']
    )
    model.summary()
    
    # 4. Callbacks
    callbacks = [
        ModelCheckpoint(MODEL_PATH, monitor='val_accuracy', save_best_only=True, verbose=1),
        EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-7, verbose=1)
    ]
    
    # 5. Train
    print("\nTraining...")
    history = model.fit(
        train_gen,
        steps_per_epoch=train_gen.n // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=val_gen,
        validation_steps=val_gen.n // BATCH_SIZE,
        callbacks=callbacks,
        class_weight=class_weights,
        verbose=1
    )
    
    # 6. Plot
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    axes[0].plot(history.history['accuracy'], label='Train')
    axes[0].plot(history.history['val_accuracy'], label='Val')
    axes[0].set_title('Accuracy')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(history.history['loss'], label='Train')
    axes[1].plot(history.history['val_loss'], label='Val')
    axes[1].set_title('Loss')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    plt.show()
    
    # 7. Save labels
    labels_dict = {i: name for i, name in enumerate(class_names)}
    with open(LABELS_PATH, 'w') as f:
        json.dump(labels_dict, f, indent=2)
    print(f"\nModel: {MODEL_PATH}")
    print(f"Labels: {LABELS_PATH}")

## 7. Download

In [ ]:
from IPython.display import FileLink

if os.path.exists(MODEL_PATH):
    display(FileLink(MODEL_PATH))
    display(FileLink(LABELS_PATH))
else:
    print("Model belum tersedia.")